<h1 align="center">Segmenting and Clustering Neighborhoods in Toronto</h1>

<h2>Part 1 - Scrape Wikipedia and create Postal Code dataframe</h2>

<p>Before getting data and start exploring it, download all dependencies that will be needed.</p>

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!conda install -c anaconda beautifulsoup4
from bs4 import BeautifulSoup # library to scrape websites

print('Libraries imported.')

Solving environment: / 

<h3>Scrape wikipedia to get postal code content</h3>

In [ ]:
res = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url, 'lxml')
my_table = soup.find('table',{'class':'wikitable'})
    
postalCode=[]
borough=[]
neighborhood=[]
for row in my_table.findAll('tr'):
    cells = row.findAll('td')
    if len(cells)==3:
        postalCode.append(cells[0].find(text=True))
        borough.append(cells[1].find(text=True))
        neighborhood.append(cells[2].find(text=True))      

<h3>Cleanse the data</h3>

In [ ]:
df = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood'])
df['PostalCode'] = postalCode
df['Borough'] = borough
df['Neighborhood'] = neighborhood

# Ignore cells with a borough that is Not assigned
df = df[df['Borough' != 'Not assigned']]

# Combine rows with same PostalCode, separate neighborhoods by comma
df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(','.join).reset_index()

# If cell has a borough but neighborhood is Not Assigned, set Neighborhood to be same as the borough.
df['Neighborhood'] = np.where(df['Borough'] != '' and df['Neighborhood'] == 'Not assigned', df['Borough'], df['Neighborhood'])

# Print shape of dataframe
df.shape